# Subject: Advanced Data Analysis

# Module: Geospatial Analysis

## Session 4 - GIS Spatial analysis and Geoprocessing 

### Demo 1 -  Find hospitals closest to an incident in Barcelona

### Overview:

The `network` module of the ArcGIS API for Python can be used to solve different types of network analysis operations. 

### You will learn: 

In this use case, we see how to find the hospital that is closest to an incident in Barcelona. We also will need to use the Geocoding and Reverse Geocoding services.

#### Option: 

You can run this application using: notebooks.esri.com.

https://notebooks.esri.com

## Closest facility

The closest facility solver provides functionality for finding out the closest locations to a particular input point. This solver would be useful in cases when you have an incident and need to find the closest facility or need to get information on the travel time and the distance to each of the facilities from an incident point for reporting purposes.

![](http://desktop.arcgis.com/en/arcmap/latest/extensions/network-analyst/GUID-96C273DB-6A24-4D42-AADA-975A33B44F3D-web.png)

When finding closest facilities, you can specify how many to find and whether the direction of travel is toward or away from them. The closest facility solver displays the best routes between incidents and facilities, reports their travel costs, and returns driving directions.

### Connect to your GIS
As a first step, you would need to establish a connection to your organization which could be an ArcGIS Online Developer account.

In [1]:
from IPython.display import HTML
import pandas as pd
from arcgis.gis import GIS

#connect to your GIS
user_name = 'Gutierres_bts'
password = 'Liberdade3030'
my_gis = GIS('https://www.arcgis.com', user_name, password)

### 1. Create a Network Layer
To perform any network analysis (such as finding the closest facility, the best route between multiple stops, or service area around a facility), you would need to create a `NetworkLayer` object. In this use case, since we are solving for closest facilities, we need to create a `ClosestFacilityLayer` which is a type of `NetworkLayer`.

To create any `NetworkLayer` object, you would need to provide the URL to the appropriate network analysis service. Hence, in this use, ESRI provide a `ClosestFacility` URL to create a `ClosestFacilityLayer` object. 

Since all ArcGIS Online Developers already have access to those routing services, you can access this URL through the `GIS` object's `helperServices` property. In future, if you have your own ArcGIS Server based map service with network analysis capability enabled, you would need to provide the URL for this service.

Let us start by importing the `network` module.

In [2]:
import arcgis.network as network

Access the analysis URL from the `GIS` object

In [3]:
analysis_url = my_gis.properties.helperServices.closestFacility.url
analysis_url

'https://route.arcgis.com/arcgis/rest/services/World/ClosestFacility/NAServer/ClosestFacility_World'

Create a `ClosestFacilityLayer` object using this URL

In [4]:
cf_layer = network.ClosestFacilityLayer(analysis_url, gis=my_gis)

In [5]:
cf_layer

<ClosestFacilityLayer url:"https://route.arcgis.com/arcgis/rest/services/World/ClosestFacility/NAServer/ClosestFacility_World">

https://route.arcgis.com/arcgis/

http://traffic.arcgis.com/arcgis/

### 2. Create hospitals layer
In this use case, we will be looking for the closest hospital (facility) to an incident location. Even though we are interested in finding out the closest one, it would still be helpful to get the information on the distance and travel time to all of them for reference purposes.

In the code below, we need to geocode the hospitals' addresses as well as do the reverse geocode for the incident location which has been supplied in the latitude/longitude format.

To perform the geocode operations, we import the `geocoding` module of the ArcGIS API.

In [6]:
from arcgis import geocoding

In this project, we geocode 4 addresses of hospitals to create the facility layer. In your workflows, this could any feature layer. Create a list of hospitals in Barcelona.

- (1) "Hospital Clínic | Hospital Clinic de Barcelona"

Address: Carrer de Villarroel, 170, 08036, Barcelona

- (2) "Hospital Sant Joan de Déu"

Address: Passeig de Sant Joan de Déu, 2, 08950, Esplugues de Llobregat, Barcelona

- (3) "Hospital del Mar" 

Address: Passeig Marítim, 25-29, 08003, Barcelona

- (4) "Hospital de la Santa Creu i Sant Pau"
Address: Carrer de Sant Quintí, 89, 08041, Barcelona

In [7]:
hospitals_addresses = ['Carrer de Villarroel, 170, 08036, Barcelona',
                       'Passeig de Sant Joan de Déu, 2, 08950, Esplugues de Llobregat, Barcelona',
                       'Passeig Marítim, 25-29, 08003, Barcelona',
                       'Carrer de Sant Quintí, 89, 08041, Barcelona']

Loop through each address and geocode it. The geocode operation returns a list of matches for each address. We pick the first result and extract the coordinates from it and construct a `Feature` object out of it. Then we combine all the `Feature`s representing the hospitals into a `FeatureSet` object.

In [8]:
from arcgis.features import Feature, FeatureSet

In [9]:
hosp_feat_list = []

for address in hospitals_addresses:
    hit = geocoding.geocode(address)[0]
    hosp_feat = Feature(geometry=hit['location'], attributes=hit['attributes'])

    hosp_feat_list.append(hosp_feat)

In [10]:
type(hosp_feat_list)

list

In [11]:
hosp_feat_list

[{"geometry": {"x": 2.15169749945174, "y": 41.38909293228112}, "attributes": {"Loc_name": "World", "Status": "M", "Score": 100, "Match_addr": "Carrer de Villarroel 170, 08036, l'Antiga Esquerra de l'Eixample, Barcelona, Catalunya", "LongLabel": "Carrer de Villarroel 170, 08036, l'Antiga Esquerra de l'Eixample, Barcelona, Catalunya, ESP", "ShortLabel": "Carrer de Villarroel 170", "Addr_type": "PointAddress", "Type": "", "PlaceName": "", "Place_addr": "Carrer de Villarroel 170, 08036, l'Antiga Esquerra de l'Eixample, Barcelona, Catalunya", "Phone": "", "URL": "", "Rank": 20, "AddBldg": "", "AddNum": "170", "AddNumFrom": "", "AddNumTo": "", "AddRange": "", "Side": "R", "StPreDir": "", "StPreType": "Carrer de", "StName": "Villarroel", "StType": "", "StDir": "", "BldgType": "", "BldgName": "", "LevelType": "", "LevelName": "", "UnitType": "", "UnitName": "", "SubAddr": "", "StAddr": "Carrer de Villarroel 170", "Block": "", "Sector": "", "Nbrhd": "l'Antiga Esquerra de l'Eixample", "District"

Construct a `FeatureSet` using each hospital `Feature`.

In [12]:
hospitals_fset = FeatureSet(features=hosp_feat_list, 
                            geometry_type='esriGeometryPoint', 
                            spatial_reference={'latestWkid': 4326})

In [13]:
type(hospitals_fset)

arcgis.features.feature.FeatureSet

In [14]:
hospitals_fset

{"features": [{"geometry": {"x": 2.15169749945174, "y": 41.38909293228112}, "attributes": {"Loc_name": "World", "Status": "M", "Score": 100, "Match_addr": "Carrer de Villarroel 170, 08036, l'Antiga Esquerra de l'Eixample, Barcelona, Catalunya", "LongLabel": "Carrer de Villarroel 170, 08036, l'Antiga Esquerra de l'Eixample, Barcelona, Catalunya, ESP", "ShortLabel": "Carrer de Villarroel 170", "Addr_type": "PointAddress", "Type": "", "PlaceName": "", "Place_addr": "Carrer de Villarroel 170, 08036, l'Antiga Esquerra de l'Eixample, Barcelona, Catalunya", "Phone": "", "URL": "", "Rank": 20, "AddBldg": "", "AddNum": "170", "AddNumFrom": "", "AddNumTo": "", "AddRange": "", "Side": "R", "StPreDir": "", "StPreType": "Carrer de", "StName": "Villarroel", "StType": "", "StDir": "", "BldgType": "", "BldgName": "", "LevelType": "", "LevelName": "", "UnitType": "", "UnitName": "", "SubAddr": "", "StAddr": "Carrer de Villarroel 170", "Block": "", "Sector": "", "Nbrhd": "l'Antiga Esquerra de l'Eixample

Lets draw our hospitals on a map

In [15]:
map1 = my_gis.map('Barcelona',8)
map1

In [16]:
map1.draw(hospitals_fset, symbol={"type": "esriSMS","style": "esriSMSCircle",
                                  "color": [0,0,255,255],"size": 8,})

Note: to change colors.

http://help.arcgis.com/en/arcgisserver/10.0/apis/rest/symbol.html#color
    
https://www.rapidtables.com/web/color/RGB_Color.html

### Create incidents layer
Similarly, let us create the incient layer

In [17]:
incident_coords = '2.19158,41.387578'
reverse_geocode = geocoding.reverse_geocode({"x": incident_coords.split(',')[0], 
                                              "y": incident_coords.split(',')[1]})

incident_feature = Feature(geometry=reverse_geocode['location'], 
                           attributes=reverse_geocode['address'])

In [39]:
incident_feature

{"geometry": {"x": 2.1917255096048422, "y": 41.38805167595008, "spatialReference": {"wkid": 4326, "latestWkid": 4326}}, "attributes": {"Match_addr": "Carrer de Villena 2-18, 08005, La Vila Ol\u00edmpica del Poblenou, Barcelona, Catalunya", "LongLabel": "Carrer de Villena 2-18, 08005, La Vila Ol\u00edmpica del Poblenou, Barcelona, Catalunya, ESP", "ShortLabel": "Carrer de Villena 2-18", "Addr_type": "StreetAddress", "Type": "", "PlaceName": "", "AddNum": "2", "Address": "Carrer de Villena 2", "Block": "", "Sector": "", "Neighborhood": "La Vila Ol\u00edmpica del Poblenou", "District": "Barcelona", "City": "Barcelona", "MetroArea": "", "Subregion": "Barcelona", "Region": "Catalunya", "Territory": "", "Postal": "08005", "PostalExt": "", "CountryCode": "ESP"}}

In [18]:
incident_fset = FeatureSet([incident_feature], geometry_type='esriGeometryPoint',
                          spatial_reference={'latestWkid': 4326})

Let us add the incident to the map

In [19]:
map1.draw(incident_fset, symbol={"type": "esriSMS","style": "esriSMSDiamond","color": [255,0,0,255], "size": 10})

## Solve for closest hospital
By default the closest facility service would return only the closest location, so we need to specify explicitly the `default_target_facility_count` parameter as well as `return_facilities`.


In [20]:
result = cf_layer.solve_closest_facility(incidents=incident_fset,
                                        facilities=hospitals_fset,
                                        default_target_facility_count=4,
                                        return_facilities=True,
                                        impedance_attribute_name='TravelTime',
                                        accumulate_attribute_names=['Kilometers','TravelTime'])

Notes:

- default_target_facility_count = default number of facilities to find.
- return_facilities = means an array of points, only returned when ClosestFacilityParameters.returnFacilities is true.
- impedance_attribute_name= Gets or sets the network attribute name to be used as the impedance attribute in analysis.
- accumulate_attribute_names = Gets or sets the collection of network attribute names to be accumulated with the analysis.

Let us inspect the result dictionary.

Note: dict.keys() method returns a dictionary view object,

In [21]:
result.keys()

dict_keys(['messages', 'routes', 'facilities'])

Let us use the `routes` dictionary to construct line features out of the routes to display on the map

In [22]:
result['routes'].keys()

dict_keys(['fieldAliases', 'geometryType', 'spatialReference', 'features'])

In [23]:
result['routes']['features'][0].keys()

dict_keys(['attributes', 'geometry'])

Construct line features out of the routes that are returned.

In [33]:
line_feat_list = []
for line_dict in result['routes']['features']:
    f1 = Feature(line_dict['geometry'], line_dict['attributes'])
    line_feat_list.append(f1)

In [34]:
routes_fset = FeatureSet(line_feat_list, 
                         geometry_type=result['routes']['geometryType'],
                         spatial_reference= result['routes']['spatialReference'])

Add the routes back to the map. The route to the closest hospital is in red

In [35]:
map1.draw(routes_fset)

## Analyze the results in a table
Since we parsed the routes as a `FeatureSet`, we can display the attributes easily as a `pandas` `DataFrame`.

In [36]:
routes_fset.df

,FacilityCurbApproach,FacilityID,FacilityRank,IncidentCurbApproach,IncidentID,Name,ObjectID,Shape_Length,Total_Kilometers,Total_Miles,Total_TravelTime,SHAPE
0,2,1,1,1,1,Carrer de Villena 2 - Location 1,1,0.062431,5.819194,3.615880,11.922334,"{'paths': [[[2.191690000000051, 41.38811000000..."
1,1,4,2,1,1,Carrer de Villena 2 - Location 4,2,0.054392,5.161340,3.207108,13.087476,"{'paths': [[[2.191690000000051, 41.38811000000..."
2,1,2,3,1,1,Carrer de Villena 2 - Location 2,3,0.175068,16.653934,10.348275,17.278289,"{'paths': [[[2.191690000000051, 41.38811000000..."
3,1,3,4,1,1,Carrer de Villena 2 - Location 3,4,0.471711,42.671649,26.514933,37.070750,"{'paths': [[[2.191690000000051, 41.38811000000..."


Let us add the hospital addresses and incident address to this table and display only the relevant columns

In [37]:
df1 = routes_fset.df
df1['facility_address'] = hospitals_addresses
df1['incident_address'] = [incident_feature.attributes['Match_addr'] for i in range(len(hospitals_addresses))]

In [38]:
df1[['facility_address','incident_address','Total_Kilometers','Total_TravelTime']]


,facility_address,incident_address,Total_Kilometers,Total_TravelTime
0,"Carrer de Villarroel, 170, 08036, Barcelona","Carrer de Villena 2-18, 08005, La Vila Olímpic...",5.819194,11.922334
1,"Passeig de Sant Joan de Déu, 2, 08950, Esplugu...","Carrer de Villena 2-18, 08005, La Vila Olímpic...",5.161340,13.087476
2,"Passeig Marítim, 25-29, 08003, Barcelona","Carrer de Villena 2-18, 08005, La Vila Olímpic...",16.653934,17.278289
3,"Carrer de Sant Quintí, 89, 08041, Barcelona","Carrer de Villena 2-18, 08005, La Vila Olímpic...",42.671649,37.070750


### Conclusion
Thus using the `network` module of the ArcGIS API for Python, you can solve for closest facilities from an incident location.

## Test another incident location and visualize the results.

    incident_coords = '2.139307,41.401963'
    